#Классификация эссе на наличие лжи и правды с BOW ГЛЕБ

In [7]:
# Для нейросети
import sys
sys.path.append('..')
from config import Config
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Activation # Слои для сети
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами
from tensorflow.keras.preprocessing.sequence import pad_sequences # Метод для работы с последовательностями

import numpy as np # Для работы с данными 

import os # Для работы с файлами

Функции для создания xTrain и xTest

Загружаю файлы, которые содержат списки соответствующие текстам преобразованным в последовательности индексов согласно частотному словарю

Загрузив их мне нужно обработать их разными структурами нейросети BOW.

Результаты работы нейросети с разными параметрами я занесу в Excel.

Загрузка файлов позволяет не тратить время и ресурсы виртуальной среды на новую сборку датасета.

In [8]:
# Загрузка данных файлов в списки 
xTrain = np.load(Config.DATASET_X_TRAIN, allow_pickle=True)
yTrain = np.load(Config.DATASET_Y_TRAIN, allow_pickle=True)
xTest = np.load(Config.DATASET_X_TEST, allow_pickle=True)
yTest = np.load(Config.DATASET_Y_TEST, allow_pickle=True)


In [9]:
# ЭТИ ПАРАМЕТРЫ МОЖНО И НУЖНО МЕНЯТЬ
# Задаём базовые параметры
xLen = 50 # Размер окна (количество слов в векторе)
step = 6 # Шаг разбиения текста на векторы

In [10]:
# для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrain.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTest.shape)
print(yTest.shape)

Размерности тренировочного набора
(67389, 50)
(67389, 2)

Размерности тестового набора
(868, 50)
(868, 2)


In [11]:
# Создаём полносвязную сеть
model = Sequential()
model.add(Dense(512, input_dim=50, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               26112     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 27,138
Trainable params: 27,138
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Обучаем сеть
history = model.fit(xTrain, 
                    yTrain, 
                    epochs=30,
                    batch_size=64,
                    validation_split=0.2)

Epoch 1/30
843/843 [==============================] - 1s 925us/step - loss: 7.9205 - accuracy: 0.9885 - val_loss: 751.9523 - val_accuracy: 7.4195e-05
Epoch 2/30
843/843 [==============================] - 1s 834us/step - loss: 2.1761 - accuracy: 0.9892 - val_loss: 262.8711 - val_accuracy: 2.9678e-04
Epoch 3/30
843/843 [==============================] - 1s 977us/step - loss: 0.8131 - accuracy: 0.9895 - val_loss: 130.0920 - val_accuracy: 0.0026
Epoch 4/30
843/843 [==============================] - 1s 1ms/step - loss: 0.4426 - accuracy: 0.9896 - val_loss: 97.5915 - val_accuracy: 0.0013
Epoch 5/30
843/843 [==============================] - 1s 989us/step - loss: 0.2913 - accuracy: 0.9893 - val_loss: 68.2403 - val_accuracy: 0.0020
Epoch 6/30
843/843 [==============================] - 1s 895us/step - loss: 0.1931 - accuracy: 0.9884 - val_loss: 50.3619 - val_accuracy: 0.0011
Epoch 7/30
843/843 [==============================] - 1s 883us/step - loss: 0.1318 - accuracy: 0.9891 - val_loss: 51.7163

In [13]:
# При базовых параметрах =
# xLen = 500 # Размер окна (количество слов в векторе)
# step = 60 # Шаг разбиения текста на векторы
# Результат
# Epoch 10/10
# 157/157 [==============================] - 14s 90ms/step - loss: 1.4291e-07 - accuracy: 1.0000 - val_loss: 5.7179e-08 - val_accuracy: 1.0000
# Когда step < xLen, н.с. обучается значительно дольше
# Результат с новыми параметрами остался прежним

Проверка работы обученной нейросети

In [14]:
# Проверка точности на тестовой выборке
rightAnswer = [0,0]
totalAnswer = [0,0]

# Получаем результаты распознавания для каждого класса
currPred = model.predict(xTest)
# Определяем номер распознанного класса для каждого вектора
currOut = np.argmax(currPred, axis=1)
# Определяем правильный класс для каждого вектора
yOut = np.argmax(yTest, axis=1)

print(currPred.shape)
print(currOut.shape)
print(yOut.shape)
print()

(868, 2)
(868,)
(868,)



In [15]:
# Подсчёт количества верных ответов
for i in range(len(yOut)):
  predictA = currOut[i]
  rightA   = yOut[i]

  totalAnswer[rightA] += 1
  if predictA == rightA:
    rightAnswer[rightA] += 1

print("Точность распознавания текстов")
for i in range(2):
  print("{:12s}: {:3d} из {:3d} - {:3.2f}%".format(str(i), rightAnswer[i], totalAnswer[i], (rightAnswer[i]/totalAnswer[i]*100)))

Точность распознавания текстов
0           : 701 из 701 - 100.00%
1           :   0 из 167 - 0.00%
